### JOINS and FOREIGN KEYS

https://www.postgresqltutorial.com/postgresql-sample-database/

#### Singer table:

In [ ]:
CREATE TABLE singer(
id SERIAL PRIMARY KEY,
name VARCHAR(40)
);

INSERT INTO singer(id, name)
VALUES
(1, 'Nicki Minaj'),
(2, 'Lady Gaga'),
(3, 'Taylor Swift'),
(4, 'Tom Jones');

#### Song table:

In [ ]:
CREATE TABLE song (
   id    INTEGER  PRIMARY KEY,
   name  CHARACTER VARYING(20),
   singer_id INTEGER,

   FOREIGN KEY(singer_id)
      REFERENCES singer(id)
   );

In [ ]:
INSERT INTO song(id, name, singer_id) 
VALUES
(1,'Anaconda',1),
(2,'Paparazzi',2),
(3,'Bad Romance',2),
(4,'Sex Bomb', 4);

In [ ]:
insert into singer(id,name)
values(5, 'Baris Manco');

insert into song(id, name, singer_id)
values (4,'Zehra',5),
(2,'Paparazzi' ,2);

DELETE from singer where id=5;

ERROR:  duplicate key value violates unique constraint "song_pkey"

You need to either recreate it with

In [ ]:
CREATE TABLE song (
   id    INTEGER  PRIMARY KEY,
   name  CHARACTER VARYING(20),
   singer_id INTEGER,

   FOREIGN KEY(singer_id)
      REFERENCES singer(id) ON DELETE CASCADE
   );

Or

In [ ]:
\d+

alter table song
drop constraint song_singer_id_fkey;

alter table song
add constraint song_singer_id_fkey
foreign key(singer_id)
REFERENCES singer(id) ON DELETE CASCADE;

DELETE from singer where id=5;

### JOINS

#### FIND COLUMN NAMES AND NUMBER OF ROWS

SELECT column_name
FROM information_schema.columns
WHERE table_name='orders';

select count(order_id) from orders;

SELECT column_name
FROM information_schema.columns
WHERE table_name='order_details';

select count(order_id) from order_details;

#### JOINS

orders has 830 rows and order_details 2105 
(because some orders have multiple lines - for different products).

SELECT orders.order_id, orders.customer_id, order_details.unit_price
FROM orders
RIGHT JOIN order_details
ON orders.order_id = order_details.order_id;

select count(order_id) from order_details;
select count(distinct order_id) from order_details;

How to check that the number of rows now is larger? 
Try it with the below view.

#### VIEWS

Logical storage of the query. In the below right and 
left join will give the same result as the order_ids are fully 
consistent in both tables. There is repetition but nothing missing.

CREATE VIEW rjoin as
SELECT orders.order_id, orders.customer_id, order_details.unit_price
FROM orders
RIGHT JOIN order_details
ON orders.order_id = order_details.order_id;

select count(order_id) from rjoin;

However in the below since the saffron_orders have only new customers 
not in the orders table, the all joins will be inconsistent.

create table saffron_orders(
order_id INT,
me_customer VARCHAR(30));

insert into saffron_orders(order_id, me_customer)
values(3,'Ugur Ural'),
(4,'Aynur Ural')                                          ;

select * from saffron_orders;

CREATE VIEW rjoin3 as
SELECT  orders.customer_id,
saffron_orders.order_id,
saffron_orders.me_customer
FROM orders
RIGHT JOIN saffron_orders
ON orders.order_id = saffron_orders.order_id;

CREATE VIEW ljoin3 as
SELECT  orders.customer_id,
saffron_orders.order_id,
saffron_orders.me_customer
FROM orders
LEFT JOIN saffron_orders
ON orders.order_id = saffron_orders.order_id;

* rjoin3 gives all from saffron_orders (which is at right), 
* ljoin3 gives all from orders (which is at left), 
* ljoin4 gives all from saffron_orders.

CREATE VIEW ljoin4 as
SELECT  orders.customer_id,
saffron_orders.order_id,
saffron_orders.me_customer
FROM saffron_orders
LEFT JOIN orders
ON orders.order_id = saffron_orders.order_id;

#### FULL JOIN

CREATE VIEW fulljoin as
SELECT  orders.customer_id,
saffron_orders.order_id,
saffron_orders.me_customer
FROM saffron_orders
FULL JOIN orders
ON orders.order_id = saffron_orders.order_id;

#### INNER JOIN

CREATE VIEW innerjoin as
SELECT  orders.customer_id,
saffron_orders.order_id,
saffron_orders.me_customer
FROM saffron_orders
INNER JOIN orders
ON orders.order_id = saffron_orders.order_id;

select count(order_id) from innerjoin ;

#### ABBREVIATIONS

orders.order_id will become o.order_id

DROP VIEW IF EXISTS rjoin;

CREATE VIEW rjoin as
SELECT o.order_id, od.unit_price
FROM orders as o
RIGHT JOIN order_details as od
ON o.order_id = od.order_id;

select count(order_id) from rjoin;

#### JOIN + VIEW + AGGREGATION + ABBREVIATION

Let’s see the customers who spent the most as well as how much 
in total they bought.

Create a view, with a group by aggregation:

Name it sales_by_customer_temp.

Get 3 columns:

* customer id from orders

* sum of unit price (per customer) from order_details

* sum of quantity(per customer) from order details

Abbreviate the table names.

Use an outer join on the order_id column.


CREATE VIEW sales_by_customer_temp6 AS
SELECT o.customer_id, sum(od.unit_price) AS price_by_customer,
sum(od.quantity) AS quantity_by_customer
FROM orders as o
FULL JOIN order_details as od
ON o.order_id = od.order_id
GROUP BY o.customer_id
ORDER BY price_by_customer DESC;